<a href="https://colab.research.google.com/github/tomaszpakula/Data-Science/blob/main/Assigment_5_optional_Tomasz_Paku%C5%82a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Finding similar movies

In this notebook we will see how find similar [movies](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset) by similarity of different features and using clustering.

In [ ]:
#!gdown 15SSBsuO9ap-kheYtsLJwBhJ8NKeVmhRr
!gdown 1hI4TOtDkUhivThGoXetFYI_iPj2jmF8i
!gdown 1XXkID9Euah2rgGur99ScH96S7eUas-Lf

Downloading...
From: https://drive.google.com/uc?id=1hI4TOtDkUhivThGoXetFYI_iPj2jmF8i
To: /content/movies_metadata.csv
100% 34.4M/34.4M [00:00<00:00, 159MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XXkID9Euah2rgGur99ScH96S7eUas-Lf
To: /content/keywords.csv
100% 6.23M/6.23M [00:00<00:00, 90.2MB/s]


In [ ]:
import pandas as pd
import numpy as np
import spacy
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
pd.reset_option('display.max_colwidth')
movies = pd.read_csv("movies_metadata.csv")
key = pd.read_csv("keywords.csv")
movies['overview'] = movies['overview'].fillna('')
movies['original title'] = movies['original_title'].drop_duplicates()
movies.head()

<ipython-input-76-303b70745cce>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,original title
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Jumanji
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpier Old Men
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Waiting to Exhale
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Part II


In [ ]:
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
movies = movies.dropna(subset=['id'])
movies['id'] = movies['id'].astype('int64')

<ipython-input-77-a9c827418ace>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['id'] = movies['id'].astype('int64')


In [ ]:
movies = movies.merge(key, on = 'id')
movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,original title,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Jumanji,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpier Old Men,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Waiting to Exhale,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Part II,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46477,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,رگ خواب,"[{'id': 10703, 'name': 'tragic love'}]"
46478,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,Siglo ng Pagluluwal,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46479,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,NaN,[]
46480,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,Satana likuyushchiy,[]


##Vectorizing overviews and computing cosine distance

To find similarity we can vectorize our overviews and use cosine similarity. It's better to use TF-IDF vectorizer rather than Count vectorizer, because some of words are common some of them not, and we need to have weightes of these words. Also it's better to use cosine similarity rather than Euclidean metrics, cause we don't want to advantage texts by their length.

It is also good to remove stopwords from text during vectorizing.

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
matrix = vectorizer.fit_transform(movies['overview'])

In [ ]:
names = vectorizer.get_feature_names_out()
print(names)

['00' '000' '000km' ... '첫사랑' 'ﬁrst' 'ﬁve']


Let choose one of movies like 'Spider-Man', and see its overview.

In [ ]:
pd.set_option('display.max_colwidth', None)
print(movies[movies['original_title']=='Spider-Man']['overview'])
index = movies[movies['original_title']=='Spider-Man'].index

5235    After being bitten by a genetically altered spider, nerdy high school student Peter Parker is endowed with amazing powers.
Name: overview, dtype: object


Now we can compute cosine distance of vectors.

\begin{align}
        cosine_{distance} = 1 -cosine_{similairty} = 1 - \frac{A*B}{||A||*||B||}
\end{align}

In [ ]:
pd.reset_option('display.max_colwidth')
movies['cosine distance'] = cosine_distances(matrix, matrix[index])
movies.sort_values(by = "cosine distance" , ascending = True).head(15)[['original_title', 'overview', 'cosine distance']]

,original_title,overview,cosine distance
5235,Spider-Man,After being bitten by a genetically altered sp...,0.000000
11825,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,0.726066
23259,The Amazing Spider-Man 2,"For Peter Parker, life is busy. Between taking...",0.767217
9383,Immortel (ad vitam),New York City in the year 2095 where genetical...,0.777267
19203,The Amazing Spider-Man,Peter Parker is an outcast high schooler aband...,0.782772
44723,Phoenix,When a mining colony is taken over by genetica...,0.785990
7941,Spider-Man 2,Peter Parker is going through a major identity...,0.797023
2595,Arachnophobia,A large spider from the jungles of South Ameri...,0.805806
1903,Gremlins 2: The New Batch,Young sweethearts Billy and Kate move to the B...,0.829025
41448,ねらわれた学園,"Mitamura Yuka is a normal, shy high school stu...",0.831046


And now we can see that the most similar texts include other Spider-Man movies and more.

##Words with higher weights

We could see which words from texts have higher weights, and which word have some texts in common.

In [ ]:
def top_words(title):
    dic = {'word': [], 'tf-idf': []}
    index  = movies[movies['original_title']==title].head(1).index
    for i in range(matrix.shape[1]):
      if matrix[index,i]>0:
        dic['word'].append(names[i])
        dic['tf-idf'].append(float(str(matrix[index,i]).split('\t')[1]))
    df = pd.DataFrame(dic)
    df.set_index('word',inplace = True)

    return df.sort_values(by='tf-idf',ascending=False)

In [ ]:
spiderman = top_words('Spider-Man')
spiderman

,tf-idf
word,
endowed,0.366774
bitten,0.326127
genetically,0.318139
nerdy,0.318139
altered,0.316127
spider,0.307373
parker,0.293067
amazing,0.268697
powers,0.237079


In [ ]:
common_words = spiderman.join(top_words('Spider-Man 3'), lsuffix='_Spider', rsuffix='_Spider3')
common_words = common_words.sort_values(by='tf-idf_Spider3', ascending=False).head(5)
common_words.fillna(0)

,tf-idf_Spider,tf-idf_Spider3
word,,
spider,0.307373,0.392281
altered,0.316127,0.201727
parker,0.293067,0.187012
peter,0.233459,0.148975
endowed,0.366774,0.000000


In [ ]:
common_words = spiderman.join(top_words('Immortel (ad vitam)'), lsuffix='_Spider', rsuffix='_Immortel')
common_words = common_words.sort_values(by='tf-idf_Immortel', ascending=False).head(5)
common_words.fillna(0)

,tf-idf_Spider,tf-idf_Immortel
word,,
genetically,0.318139,0.352278
altered,0.316127,0.350050
endowed,0.366774,0.000000
bitten,0.326127,0.000000
nerdy,0.318139,0.000000


We see that there are only 4 common words between Spider-man and Spider-Man 3, 2 words between Spider-man and Immortel. Let see what about Iron Man, which is simillar, but not in the top similar overviews here.

In [ ]:
common_words = spiderman.join(top_words('Iron Man'), lsuffix='_Spider', rsuffix='_Iron')
common_words = common_words.sort_values(by='tf-idf_Iron', ascending=False)
common_words.fillna(0)

,tf-idf_Spider,tf-idf_Iron
word,,
endowed,0.366774,0.0
bitten,0.326127,0.0
genetically,0.318139,0.0
nerdy,0.318139,0.0
altered,0.316127,0.0
spider,0.307373,0.0
parker,0.293067,0.0
amazing,0.268697,0.0
powers,0.237079,0.0


We see that there is no common words between these two movies. But we have more features in this dataset like genres, so we can try to represent them as vectors and compute cosine distances between these vectors.

##Similarity by genres

In [ ]:
movies['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

We have genres stored as a string of list of dictionaries, but we want to have string of genres splitted by spaces to vectorized easily. We can use literal_eval from ast library to get python list, and then take just names of genres from each dictionary.

In [ ]:
def convert(row):
  text = ""
  for dict in row:
    text += "".join(dict['name']) + " "
  return text

In [ ]:
movies['genres'] = movies['genres'].apply(literal_eval).apply(convert)

Now we can vectorize genres and compute cosine distances

In [ ]:
count_vectorizer = CountVectorizer(stop_words='english')
count_matrix = count_vectorizer.fit_transform(movies['genres'])

In [ ]:
movies['cosine distance genres'] = cosine_distances(count_matrix, count_matrix[index])
movies.sort_values(by = ["cosine distance genres",'cosine distance'] , ascending = True).head(15)[['original_title', 'overview', 'genres', 'cosine distance']]

,original_title,overview,genres,cosine distance
5235,Spider-Man,After being bitten by a genetically altered sp...,Fantasy Action,0.000000
30893,Avengers Grimm,When Rumpelstiltskin destroys the Magic Mirror...,Action Fantasy,0.883388
11222,笑傲江湖II東方不敗,"Jet Li plays Ling, a student from the Blue Mou...",Action Fantasy,0.952675
19473,Deathstalker,The warrior Deathstalker is tasked by an old w...,Action Fantasy,0.976835
35864,Chandu on the Magic Island,The second of two feature films edited from th...,Fantasy Action,0.984887
35865,Chandu on the Magic Island,The second of two feature films edited from th...,Fantasy Action,0.984887
43,Mortal Kombat,For nine generations an evil sorcerer has been...,Action Fantasy,1.000000
585,Batman,The Dark Knight of Gotham City begins his war ...,Fantasy Action,1.000000
1337,Batman Returns,"Having defeated the Joker, Batman now faces th...",Action Fantasy,1.000000
4802,The Beastmaster,"Dar, is the son of a king, who is hunted by a ...",Action Fantasy,1.000000


We see that we have different results now, and we do not have here other parts of Spider-Man, but for example we have Batman which is also similar movie. But the problem is, we didn't get Spider-Man movies. We can see why.

In [ ]:
movies[movies['title'].isin(['Spider-Man', 'Spider-Man 3'])][['title', 'genres']]

,title,genres
5235,Spider-Man,Fantasy Action
11825,Spider-Man 3,Fantasy Action Adventure


We see the problem is that we have one additional genre in Spider-Man 3, so we rather choose movies with only Fantasy and Action than with additional Adventure, cause we search the most similar vectors. Maybe it could be better to use the keywords instead of the genres.

##Similarity by keywords

At first we will do what we did for genres, and see how it looks, then vectorize it, and compute distances.

In [ ]:
movies['keywords'] = movies['keywords'].apply(literal_eval).apply(convert)

In [ ]:
movies['keywords']

0        jealousy toy boy friendship friends rivalry bo...
1        board game disappearance based on children's b...
2        fishing best friend duringcreditsstinger old men 
3        based on novel interracial relationship single...
4        baby midlife crisis confidence aging daughter ...
                               ...                        
46477                                         tragic love 
46478                                   artist play pinoy 
46479                                                     
46480                                                     
46481                                                     
Name: keywords, Length: 46482, dtype: object

In [ ]:
count_vectorizer = CountVectorizer(stop_words='english')
keywords_matrix = count_vectorizer.fit_transform(movies['keywords'])

In [ ]:
movies['cosine distance keywords'] = cosine_distances(keywords_matrix, keywords_matrix[index])
movies.sort_values(by = ["cosine distance keywords",'cosine distance'] , ascending = True).head(15)[['title', 'overview', 'keywords']]

,title,overview,keywords
5235,Spider-Man,After being bitten by a genetically altered sp...,loss of lover spider thanksgiving bad boss hos...
23136,Avengers Confidential: Black Widow & Punisher,When the Punisher takes out a black-market wea...,marvel comic superhero based on comic
30957,Iron Man & Captain America: Heroes United,Iron Man and Captain America battle to keep th...,marvel comic superhero based on comic
30809,Next Avengers: Heroes of Tomorrow,The children of the Avengers hone their powers...,marvel comic
42267,"Marvel: 75 Years, From Pulp to Pop!",In celebration of the publisher's 75th anniver...,marvel comic
21054,Captain America II: Death Too Soon,A government agent is looking for a missing sc...,marvel comic superhero captain america
26631,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,marvel comic sequel superhero based on comic v...
6224,X2,Professor Charles Xavier and his team of genet...,mutant marvel comic superhero based on comic s...
3682,X-Men,"Two mutants, Rogue and Wolverine, come to a pr...",mutant marvel comic superhero based on comic s...
4524,The Punisher,The avenging angel of Marvel Comics fame comes...,loss of family vigilante marvel comic based on...


Now we have movies, which are similar, cause they all are based on marvel comics.

In [ ]:
movies[movies['original_title']=='Spider-Man 3']['genres']

11825    Fantasy Action Adventure 
Name: genres, dtype: object

##Clustering with KMeans model

We can also divide movies by clusters using for example Kmeans model using genres and keywords.

In [ ]:
model = KMeans(n_clusters = 20)

In [ ]:
model.fit_transform(keywords_matrix, count_matrix)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([[ 4.57630806, 10.04987562,  4.71600182, ...,  4.74556886,
         4.75642317,  4.69619876],
       [ 3.3138534 ,  9.53939201,  3.50287197, ...,  3.54442104,
         3.5865159 ,  3.43200995],
       [ 2.44508247,  9.38083152,  2.71569288, ...,  2.74475938,
         2.82189586,  2.66683813],
       ...,
       [ 0.08870714,  9.05538514,  1.19205117, ...,  1.26600204,
         1.47074684,  1.0742534 ],
       [ 0.08870714,  9.05538514,  1.19205117, ...,  1.26600204,
         1.47074684,  1.0742534 ],
       [ 0.08870714,  9.05538514,  1.19205117, ...,  1.26600204,
         1.47074684,  1.0742534 ]])

In [ ]:
movies['cluster'] = model.labels_

In [ ]:
movies[movies['cluster']==5][['original_title', 'overview', 'genres','keywords']].head()

,original_title,overview,genres,keywords
312,Stargate,"An interstellar teleportation device, found in...",Action Adventure Science Fiction,space travel teleportation uprising shot to de...
903,2001: A Space Odyssey,Humanity finds a mysterious object buried bene...,Science Fiction Mystery Adventure,moon jupiter artificial intelligence man vs ma...
1163,The Empire Strikes Back,"The epic saga continues as Luke Skywalker, in ...",Adventure Action Science Fiction,rebel android asteroid space battle snow storm...
1167,Aliens,When Ripley's lifepod is found by a salvage cr...,Horror Action Thriller Science Fiction,android extraterrestrial technology space mari...
1176,Return of the Jedi,As Rebel leaders map their strategy for an all...,Adventure Action Science Fiction,rebel brother sister relationship emperor spac...


In [ ]:
movies[movies['cluster']==11][['title', 'overview', 'genres', 'keywords']].head()

,title,overview,genres,keywords
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Comedy Drama Romance,based on novel interracial relationship single...
16,Sense and Sensibility,"Rich Mr. Dashwood dies, leaving his second wif...",Drama Romance,bowling based on novel servant country life ja...
20,Get Shorty,Chili Palmer is a Miami mobster who gets sent ...,Comedy Thriller Crime,gambling miami based on novel job murder trave...
70,Fair Game,Max Kirkpatrick is a cop who protects Kate McQ...,Action Thriller Romance,bomb miami sex detective handcuffs based on no...
136,The Neon Bible,"While on a train, a teenage boy thinks about h...",Drama,southern usa based on novel singer south


We see that cluster 5 contains movies related with space. Cluster 11 contains romance drama, comedies, comedie thrillers so it looks like this model divide movies right.

##Resume

In this notebook I showed how we can find similar movies using text vectorizing, cosine similarity of vector, and KMeans model. We see also that we can use different feature like overview, genres, keywords, to find similar movies.